In [67]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [68]:
num_nodes=1000
num_edges=5000
delta=0.02
topologies=['RN','BAN']
l0_values=range(1,7)
max_samples=1000

In [69]:
def generate_topology(topology, num_nodes, num_edges):
    G=nx.Graph()
    pos={}
    if topology == 'RN':
        p=2*num_edges/(num_nodes*(num_nodes-1))
        G=nx.erdos_renyi_graph(num_nodes,p)
    elif topology == 'BAN':
        m=num_edges//num_nodes
        G=nx.barabasi_albert_graph(num_nodes,m)
    G.remove_nodes_from(list(nx.isolates(G)))
    for u,v in G.edges():
        G[u][v]['c_e']=np.random.uniform(0.97,0.99)
    return G

In [70]:
def purification_for_3paths(l0_p, l1_p, l2_p, delta):
    numerator = (1 - (1/6)*(3*l0_p + 3*l1_p + 4*l2_p)*delta +
                (1/18)*(4*l0_p*l1_p + 5*l1_p*l2_p + 5*l2_p*l0_p)*delta**2 -
                (7/54)*l0_p*l1_p*l2_p*delta**3)
    denominator = (1 - (1/6)*(3*l0_p + 3*l1_p + 2*l2_p)*delta +
                  (1/18)*(6*l0_p*l1_p + 4*l1_p*l2_p + 4*l2_p*l0_p)*delta**2 -
                  (4/27)*l0_p*l1_p*l2_p*delta**3)
    return numerator / denominator if denominator != 0 else 0.0

In [71]:
def purification(C1, C2):
    return (13 + 14*C1 + 14*C2 + 40*C1*C2) / (41 + 4*C1 + 4*C2 + 32*C1*C2)

In [72]:
def find_3_shortest_paths(G, s, d):
    subgraph = G.copy()
    paths = []
    path_lengths = []
    for i in range(1, 4):
        try:
            path = nx.shortest_path(subgraph, source=s, target=d)
            paths.append(path)
            path_lengths.append(len(path) - 1)
            for u, v in zip(path[:-1], path[1:]):
                subgraph.remove_edge(u, v)
        except nx.NetworkXNoPath:
            break
    paths = sorted(paths, key=len, reverse=True)
    return paths, path_lengths

In [73]:
def find_node_pairs_with_path_length(G, target_length):
    node_pairs = []
    for node1 in tqdm(G.nodes):
        for node2 in G.nodes:
            if node1 != node2:
                try:
                    path_length = nx.shortest_path_length(G, source=node1, target=node2)
                    if path_length == target_length:
                        node_pairs.append((node1, node2))
                except nx.NetworkXNoPath:
                    pass
    return node_pairs

In [74]:
def Concurrence_after_swapping(path, G):
    C=[]
    for u, v in zip(path[:-1], path[1:]):
        C.append(G[u][v]['c_e'])
    if len(C)>1:
        C1=C[0]
        for i in range(len(C)-1):
            C2=C[i+1]
            C3=(C1 + C2 + 2*C1*C2 - 1) / 3
            C1=C3
    else:
        C3=C[0]
    return C3

In [75]:
def Concurrence_after_purification(paths, G):
    C_path=[]
    for path in paths:
        C_path.append(Concurrence_after_swapping(path, G))
    if len(C_path)>1:
        C_path1=C_path[0]
        for i in range(len(C_path)-1):
            C_path2=C_path[i+1]
            C_path3=purification(C_path1, C_path2)
            C_path1=C_path3
    else:
        C_path3=C_path[0]
    return C_path3

In [76]:
def Concurrence_after_swapping_and_purification(G, s, d):
    paths, path_lengths = find_3_shortest_paths(G, s, d)
    return Concurrence_after_purification(paths, G)

In [ ]:
results = {topo: {l0: {'MEP': 0.0, 'Link-level MEP': 0.0} for l0 in l0_values} for topo in topologies}
for topology in topologies:
    print(f"Processing {topology}")
    G = generate_topology(topology, num_nodes, num_edges)
    for l0 in l0_values:
        print(f"\nProcessing l0 = {l0}")
        node_pairs = find_node_pairs_with_path_length(G,l0)
        num_of_node_pairs = len(node_pairs)
        np.random.shuffle(node_pairs)
        if num_of_node_pairs > max_samples :
            node_pairs = node_pairs[:1000]
        C_sp_total = 0
        C_ps_total = 0
        for s, d in tqdm(node_pairs):
            C_sp = Concurrence_after_swapping_and_purification(G, s, d)
            C_sp_total = C_sp_total + C_sp
            S_path = nx.shortest_path(G, source=s, target=d)
            C_ll = []
            for m, n in zip(S_path[:-1], S_path[1:]):
                C_ll.append(Concurrence_after_swapping_and_purification(G, m, n))
            if len(C_ll)>1:
                C_ll1 = C_ll[0]
                for i in range(len(C_ll)-1):
                    C_ll2 =C_ll[i+1]
                    C_ll3=(C_ll1+C_ll2+2*C_ll1*C_ll2-1)/3
                    C_ll1=C_ll3
            else:
                C_ll3=C_ll[0]
            C_ps_total = C_ps_total + C_ll3
        if len(node_pairs) == 0:
            C_ps_avg = 0
            C_sp_avg = 0
        else:
            C_ps_avg = C_ps_total / len(node_pairs)
            C_sp_avg = C_sp_total / len(node_pairs)
        results[topology][l0]['MEP'] = C_sp_avg
        results[topology][l0]['Link-level MEP'] = C_ps_avg

# Plotting
plt.figure(figsize=(10,6))
markers = ['o', 's']
colors = ['gold', 'sienna']
labels = ['RN', 'BAN']
for idx, topo in enumerate(topologies):
    x = list(l0_values)
    y_mep = [results[topo][l0]['MEP'] for l0 in l0_values]
    y_llmep = [results[topo][l0]['Link-level MEP'] for l0 in l0_values]
    plt.plot(x, y_mep, marker=markers[idx], color=colors[idx], linestyle='-', label=f'{labels[idx]} MEP')
    plt.plot(x, y_llmep, marker=markers[idx], color=colors[idx], linestyle=':', label=f'{labels[idx]} Link-level MEP')
plt.xlabel('Shortest Path Length (l₀)')
plt.ylabel('Average Concurrence')
plt.title('Multipath Entanglement Purification Strategies')
plt.legend()
plt.grid(True)
plt.ylim(0.88, 1.00)
plt.show()

Processing RN

Processing l0 = 1


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:25<00:00, 11.75it/s]



Processing l0 = 2


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:09<00:00,  7.70it/s]



Processing l0 = 3


...

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:56<00:00,  8.62it/s]



Processing l0 = 3


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:43<00:00,  6.13it/s]



Processing l0 = 4


  1%|▋                                                                                | 9/1000 [00:00<00:46, 21.37it/s]